## Start of our student loan balance analysis

In [11]:
# Install packages
install.packages(c('janitor','car'))

Warning message:
"packages 'janitor', 'car' are in use and will not be installed"


## Data Cleaning

In [ ]:
library(dplyr)
library(janitor)
library(purrr)
library(glue)
library(stringr)
library(readxl)

load_and_combine_data <- function(directory_path) {
  
  file_pattern <- glue(".*-Data\\.csv$")
  
  file_paths <- list.files(path = directory_path, pattern = file_pattern, full.names = TRUE)
  
  list_of_dfs <- map(file_paths, function(path) { 
    
    year_match_result <- regmatches(path, regexpr("Y(\\d{4})\\.*", path))
    
    year_extracted <- sub("Y(\\d{4})\\.*", "\\1", year_match_result)
    
    read.csv(path, skip = 1, na.strings = c("N", "-", "(X)", "null")) %>% 
      clean_names() %>%
      mutate(year = as.numeric(year_extracted)) %>%
      mutate(state = str_trim(str_extract(geographic_area_name, ",\\s*([^,]+)$"), side = "left"),
             state = str_replace(state, "^,\\s*", "")) %>%
      select(-contains(c("margin_of_error", "ratio", "unit", "percent", "x", "geograph")))
      }
    )
  
  common_columns <- names(list_of_dfs[[1]])
  
  for (i in 2:length(list_of_dfs)) {
    common_columns <- intersect(common_columns, names(list_of_dfs[[i]]))
  }
  
  list_of_dfs_common_cols <- map(list_of_dfs, ~ .x %>% select(all_of(common_columns)))
  
  for (col_name in common_columns) {
    first_col_type <- class(list_of_dfs_common_cols[[1]][[col_name]])
    
    if (first_col_type %in% c("numeric", "integer")) {
      list_of_dfs_common_cols <- map(list_of_dfs_common_cols, function(df) {
        df[[col_name]] <- as.numeric(df[[col_name]])
        df
      })
    } else if (first_col_type == "factor") {
      list_of_dfs_common_cols <- map(list_of_dfs_common_cols, function(df) {
        df[[col_name]] <- as.character(df[[col_name]])
        df
      })
    }
  }
  
  yearly_data <- bind_rows(list_of_dfs_common_cols)
  
  yearly_data
}

load_and_combine_borrower_data <- function(filepath, sheet_numbers, years) {

  combined_borrower_data <- map2_df(sheet_numbers, years, function(sheet, year) {
    read_excel(filepath, sheet = sheet, skip = 7) %>%
    clean_names() %>%
    mutate(year = year)
  })
  
  final_borrower_data <- combined_borrower_data %>%
    select(state, year, total_borrowers, total_balance_billions)
  
  final_borrower_data
}

dem_data <- load_and_combine_data("../data/dem") %>%
  rename(total_population = estimate_race_total_population,
         race_white = estimate_race_total_population_one_race_white,
         race_black = estimate_race_total_population_one_race_black_or_african_american,
         race_asian = estimate_race_total_population_one_race_asian,
         race_native = estimate_race_total_population_one_race_american_indian_and_alaska_native,
         race_pacific_islander = estimate_race_total_population_one_race_native_hawaiian_and_other_pacific_islander,
         race_other = estimate_race_total_population_one_race_some_other_race,
         race_two_or_more = estimate_race_total_population_two_or_more_races,
         hispanic_or_latino = estimate_hispanic_or_latino_and_race_total_population_hispanic_or_latino_of_any_race,
         total_18_or_over_population = estimate_citizen_voting_age_population_citizen_18_and_over_population,
         population_18_or_over_male = estimate_citizen_voting_age_population_citizen_18_and_over_population_male,
         population_18_or_over_female = estimate_citizen_voting_age_population_citizen_18_and_over_population_female) %>%
  select(-contains("estimate")) %>%
  group_by(state, year) %>%
  summarise(across(everything(), ~sum(., na.rm=TRUE))) %>%
  ungroup() %>%
  mutate(across(race_two_or_more:hispanic_or_latino, ~./total_population),
         across(population_18_or_over_male:population_18_or_over_female, ~./total_18_or_over_population))

economic_data <- load_and_combine_data("../data/economic") %>%
  rename(employment_total = estimate_employment_status_population_16_years_and_over,
         employement_total_employed = estimate_employment_status_population_16_years_and_over_in_labor_force,
         employment_total_unemployed = estimate_employment_status_population_16_years_and_over_not_in_labor_force) %>%
  select(-contains("estimate"))  %>%
  group_by(state, year) %>%
  summarise(across(everything(), ~sum(., na.rm=TRUE))) %>%
  ungroup() %>%
  mutate(across(employement_total_employed:employment_total_unemployed, ~./employment_total)) %>%
  select(-employment_total)

mean_income_data <- load_and_combine_data("../data/mean_income") %>%
  group_by(state, year) %>%
  summarise(mean_household_income_dollars = mean(estimate_mean_income_dollars_household_income_all_households, na.rm=TRUE)) %>%
  ungroup()

social_char_data <- load_and_combine_data("../data/social_char") %>%
  rename(average_household_size = estimate_households_by_type_total_households_average_household_size,
         total_veteran_eligible = estimate_veteran_status_civilian_population_18_years_and_over,
         total_veteran = estimate_veteran_status_civilian_population_18_years_and_over_civilian_veterans,
         total_population_degree = estimate_educational_attainment_population_25_years_and_over,
         total_population_bachelors_degree = estimate_educational_attainment_population_25_years_and_over_bachelor_s_degree_or_higher,
         total_foreign_born_pop = estimate_u_s_citizenship_status_foreign_born_population,
         total_households = estimate_households_by_type_total_households,
         total_household_w_internet = estimate_computers_and_internet_use_total_households_with_a_broadband_internet_subscription) %>%
  select(-contains("estimate")) %>%
  group_by(state, year) %>%
  summarise(across(-average_household_size, ~sum(., na.rm=TRUE)), 
            average_household_size = mean(average_household_size, na.rm=TRUE)) %>%
  ungroup() %>%
  mutate(total_veteran = total_veteran/total_veteran_eligible, 
         total_population_bachelors_degree = total_population_bachelors_degree/total_population_degree,
         total_household_w_internet = total_household_w_internet/total_households) %>%
  select(-c(total_veteran_eligible, total_population_degree, total_households))

borrower_data <-load_and_combine_borrower_data("../data/Student-loan-update-2025-Mangrum.xlsx", 11:15, 2019:2023)

combined_data <- dem_data %>%
  left_join(economic_data, by = c("state", "year")) %>%
  left_join(mean_income_data, by = c("state", "year")) %>%
  left_join(social_char_data, by = c("state", "year")) %>%
  left_join(borrower_data, by = c("state", "year")) %>%
  relocate(state, year) %>%
  filter(!state == "Puerto Rico", !state == "District of Columbia") %>%
  mutate(total_foreign_born_pop = total_foreign_born_pop/total_population,
         total_borrowers = total_borrowers/total_18_or_over_population) %>%
  select(-c(total_population, total_18_or_over_population))

# combined_data %>%
#   select(-state, -year) %>%
#   gtsummary::tbl_summary()

write.csv(combined_data, "../data/combined_clean_data.csv", row.names = FALSE)




`summarise()` has grouped output by 'state'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'state'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'state'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'state'. You can override using the
`.groups` argument.


In [12]:
# Scratch work - MLR
set.seed(123)
library(dplyr)
library(janitor)
library(purrr)
library(glue)
library(stringr)
library(readxl)
library(glmnet)
library(car)
student_debt_data <- read.csv("../data/combined_clean_data.csv")

train_idx <- sample(seq_len(nrow(student_debt_data)), size = floor(0.8 * nrow(borrower_data)))
train_data <- student_debt_data[train_idx, ]
test_data <- student_debt_data[-train_idx, ]
# Prep train data
train_data <- train_data %>%
  mutate(minority_share = 1 - race_white) %>%
  select(-starts_with("race_")) %>%
  select(-state, -year, -population_18_or_over_female, -employment_total_unemployed)
# Prepare test data
test_data <- test_data %>%
  mutate(minority_share = 1 - race_white) %>%
  select(-starts_with("race_")) %>%
  select(-state, -year, -population_18_or_over_female, -employment_total_unemployed)
colnames(test_data)
colnames(train_data)
#mlr_model <- lm(total_balance_billions ~ . -year, data = train_data)
mlr_model <- lm(total_balance_billions ~ ., data = train_data)

summary(mlr_model)
vif(mlr_model)

[1] "hispanic_or_latino"                "population_18_or_over_male"       
 [3] "employement_total_employed"        "mean_household_income_dollars"    
 [5] "total_population_bachelors_degree" "total_veteran"                    
 [7] "total_foreign_born_pop"            "total_household_w_internet"       
 [9] "average_household_size"            "total_borrowers"                  
[11] "total_balance_billions"            "minority_share"

[1] "hispanic_or_latino"                "population_18_or_over_male"       
 [3] "employement_total_employed"        "mean_household_income_dollars"    
 [5] "total_population_bachelors_degree" "total_veteran"                    
 [7] "total_foreign_born_pop"            "total_household_w_internet"       
 [9] "average_household_size"            "total_borrowers"                  
[11] "total_balance_billions"            "minority_share"


Call:
lm(formula = total_balance_billions ~ ., data = train_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-53.569 -13.607  -1.184  10.648  69.146 

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                       -1.477e+02  1.092e+02  -1.352 0.177735    
hispanic_or_latino                 3.823e+00  2.036e+01   0.188 0.851224    
population_18_or_over_male         1.750e+02  2.607e+02   0.671 0.502828    
employement_total_employed        -1.713e+02  7.755e+01  -2.208 0.028319 *  
mean_household_income_dollars     -1.253e-03  1.760e-04  -7.117 1.75e-11 ***
total_population_bachelors_degree  2.458e+01  5.544e+01   0.443 0.658001    
total_veteran                     -5.715e+02  1.258e+02  -4.542 9.44e-06 ***
total_foreign_born_pop             3.908e+02  4.760e+01   8.209 2.32e-14 ***
total_household_w_internet         2.299e+02  5.929e+01   3.878 0.000141 ***
average_household_size             1.462e+01  1.170e+0

hispanic_or_latino        population_18_or_over_male 
                         2.436188                          4.356888 
       employement_total_employed     mean_household_income_dollars 
                         3.909949                          4.037348 
total_population_bachelors_degree                     total_veteran 
                         4.441363                          2.041249 
           total_foreign_born_pop        total_household_w_internet 
                         4.594095                          2.872773 
           average_household_size                   total_borrowers 
                         1.454985                          2.739081 
                   minority_share 
                         2.248205